In [1]:
import pandas as pd
from os.path import join 
from core.data import Dataset
from core.configs import *
from core.train import Train
import optuna
import plotly.express as px
import mlflow
%cd ~/notebooks/petrol/
mlflow.set_tracking_uri('mlruns')

/home/prog3/notebooks/petrol


/home/prog3/.cache/pypoetry/virtualenvs/petrol-j8nzVRxz-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/prog3/.cache/pypoetry/virtualenvs/petrol-j8nzVRxz-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# 2022 - 1250

## Объединение с влагомером и водой

In [ ]:
df = Dataset(verbose=True)
df.load(join(DATA, 'raw', '2022', 'Режимный лист МУИС 2022.xlsx1250_clear.csv'), parse_dates=['Дата'])
df.drop(['Unnamed: 0'])

df.df.columns = [
 "Date","BlockP","BlockT",
 "C1P","C1T","C1L",
 "C2P","C2T","C2interfacial_L","С2petrol_L",

 "OH1T","OH1interfacial_L","OH1P",
 
 "C3T","C3L",
 "P1Tinput","P1Toutput","P1Pinput","P1Poutput","P1Тcoolant","P1Tgases","P1Рburner",
 "P2Tinput","P2Toutput","P2Pinput","P2Poutput","P2Tcoolant","P2Tgases","P2Pburner",
 "BEV1L","BEV1V_water","BEV1V_petrol","BEV1T_water"
]


vlagomer = pd.read_csv(join(DATA, 'raw', '2022', '1250_2022_vlagomer.csv'), parse_dates=['Дата']).drop(columns='Unnamed: 0')
vlagomer.columns = ['Date', 'Water_2hours', 'Vlagomer']

df.df = df.df.merge(vlagomer)
df.save(join(DATA, 'raw', '2022','2022_1250_joined_vlagomer.csv'))
df.df

## Подготовка к train

In [ ]:
cols_to_drop = ['BEV1L','BEV1V_water', 'BEV1V_petrol', 'BEV1T_water']
df_2022 = Dataset(verbose=False)
df_2022.load(join(DATA, 'preprocessed', '2022_1250_joined_vlagomer.csv'), parse_dates=['Date'])
df_2022.drop(cols_to_drop)
df_2022.astype(df_2022.cols[1:], 'float32')
df_2022.corr_matrix(df_2022.cols[1:], target='Vlagomer', show=True, height=400, width=1800, title='До скалирования/замены выбросов/minmax скалирования')
df_2022.scale(df_2022.cols[1:], method='standard')
df_2022.recovery_outliers(df_2022.cols[1:-1], save_interval=[-6,10], insert=False)
df_2022.scale(df_2022.cols[1:], method='minmax')


df_2022.corr_matrix(df_2022.cols[1:], target='Vlagomer', show=True, height=400, width=1800, title='До скалирования/замены выбросов/minmax скалирования')
# df_2022.time_series(df_2022.cols[1:], time_axis='Date', show=True, height=500, width=1750)
df_2022.save(join(DATA, 'preprocessed', '2022_1250_scaled.csv'))

### LightGBM

#### Все признаки:
    'BlockP', 'BlockT', 'C1P', 'C1T', 'C1L', 'C2P', 'C2T',
    'C2interfacial_L', 'С2petrol_L', 'OH1T', 'OH1interfacial_L', 'OH1P',
    'C3T', 'C3L', 'P1Tinput', 'P1Toutput', 'P1Pinput', 'P1Poutput',
    'P1Тcoolant', 'P1Tgases', 'P1Рburner', 'P2Tinput', 'P2Toutput',
    'P2Pinput', 'P2Poutput', 'P2Tcoolant', 'P2Tgases', 'P2Pburner', 'Water'

In [2]:
df_2022 = Dataset(name='2022_1250_scaled',verbose=False)
df_2022.load(join(DATA, 'preprocessed', '2022_1250_scaled.csv'), parse_dates=['Date'])

train = Train(df_2022, test_size=0.2)
train.optimize('lightgbm_2022_1250_scaled', 100) # дашборд запускается $ optuna-dashboard sqlite:///db.sqlite3
# train.try_all_trials(100)
train.train_with_trial(train.best_trials[0], 1500)
# train.plot_preds()

[I 2024-04-25 10:53:21,412] A new study created in RDB with name: lightgbm_2022_1250_scaled
/home/prog3/notebooks/petrol/core/train.py:126: ExperimentalWarning: set_metric_names is experimental (supported from v3.2.0). The interface can change in the future.
  study.set_metric_names(self.metrics_list)
[I 2024-04-25 10:53:22,556] Trial 0 finished with values: {'r2': 0.7118425774276733, 'mae': 0.06894150111832563, 'mse': 0.008224453108799885, 'rmse': 0.09068877057717722} and parameters: {'n_estimators': 100, 'learning_rate': 0.0160335917944502, 'num_leaves': 186, 'subsample': 0.1792219207921174, 'colsample_bytree': 0.5681536889877916, 'min_data_in_leaf': 13}. 
[I 2024-04-25 10:53:25,256] Trial 1 finished with values: {'r2': 0.7761216299624762, 'mae': 0.05843710568892213, 'mse': 0.006389830739085017, 'rmse': 0.07993641685167667} and parameters: {'n_estimators': 100, 'learning_rate': 0.16910220494880127, 'num_leaves': 940, 'subsample': 0.7796302842724114, 'colsample_bytree': 0.594892339378

{'n_estimators': 1500, 'learning_rate': 0.026956697523389854, 'num_leaves': 410, 'subsample': 0.8850913452197978, 'colsample_bytree': 0.7702514421013131, 'min_data_in_leaf': 3, 'objective': 'regression', 'metrics': ['r2', 'mae', 'mse', 'rmse'], 'verbosity': -1, 'bagging_freq': 1, 'random_state': 2024, 'device_type': 'gpu', 'force_col_wise': True}


Successfully registered model 'lightgbm_2022_1250_scaled'.
Created version '1' of model 'lightgbm_2022_1250_scaled'.


{'r2': 0.8001900003713173, 'mae': 0.05521704093393256, 'mse': 0.005702882674149937, 'rmse': 0.07551743291551916}
interval columns not set, guessing: ['BlockP', 'BlockT', 'C1P', 'C1T', 'C1L', 'C2P', 'C2T', 'C2interfacial_L', 'С2petrol_L', 'OH1T', 'OH1interfacial_L', 'OH1P', 'C3T', 'C3L', 'P1Tinput', 'P1Toutput', 'P1Pinput', 'P1Poutput', 'P1Тcoolant', 'P1Tgases', 'P1Рburner', 'P2Tinput', 'P2Toutput', 'P2Pinput', 'P2Poutput', 'P2Tcoolant', 'P2Tgases', 'P2Pburner', 'Water', 'Vlagomer']


#### Признаки: 
       'BlockP', 'BlockT', 'C1P', 'C1T', 'C1L', 'P1Tinput',
       'P1Toutput', 'P1Pinput', 'P1Poutput', 'P1Тcoolant', 'P1Tgases',
       'P1Рburner', 'P2Tinput', 'P2Toutput', 'P2Pinput', 'P2Poutput',
       'P2Tcoolant', 'P2Tgases', 'P2Pburner'

In [ ]:
df_2022.drop(['C2P', 'C2T',
       'C2interfacial_L', 'С2petrol_L', 'OH1T', 'OH1interfacial_L', 'OH1P',
       'C3T', 'C3L','Water'])

In [ ]:
train = Train(df_2022)
train.optimize('lightgbm_2022_1250_scaled_до_печек_вкл', 100) # дашборд запускается $ optuna-dashboard sqlite:///db.sqlite3
train.try_all_trials(100)


#### Признаки: 
    'BlockP', 'BlockT', 'C1P', 'C1T', 'C1L'


In [ ]:
df_2022.drop(['P1Tinput',
       'P1Toutput', 'P1Pinput', 'P1Poutput', 'P1Тcoolant', 'P1Tgases',
       'P1Рburner', 'P2Tinput', 'P2Toutput', 'P2Pinput', 'P2Poutput',
       'P2Tcoolant', 'P2Tgases', 'P2Pburner'])

In [ ]:
train = Train(df_2022)
train.optimize('lightgbm_2022_1250_блок_и_сепаратор', 100) # дашборд запускается $ optuna-dashboard sqlite:///db.sqlite3
train.try_all_trials(100)

### CatBoost

In [3]:
df_2022 = Dataset(name='2022_1250_scaled', verbose=False)
df_2022.load(join(DATA, 'preprocessed', '2022_1250_scaled.csv'), parse_dates=['Date'])

train = Train(df_2022,  model_name='catboost', metrics=['R2','MAE', 'RMSE'], directions=['maximize', 'minimize','minimize'], test_size=0.2)
train.optimize('catboost_2022_1250_scaled', 100) # дашборд запускается $ optuna-dashboard sqlite:///db.sqlite3
# train.try_all_trials(100)
train.train_with_trial(train.best_trials[0], 1500)
# train.plot_preds()

[I 2024-04-25 10:56:22,445] A new study created in RDB with name: catboost_2022_1250_scaled


/home/prog3/notebooks/petrol/core/train.py:126: ExperimentalWarning:

set_metric_names is experimental (supported from v3.2.0). The interface can change in the future.

[I 2024-04-25 10:56:22,731] Trial 0 finished with values: {'R2': 0.12867381886397122, 'MAE': 0.13015018769637846, 'RMSE': 0.1576990186630969} and parameters: {'n_estimators': 137, 'learning_rate': 0.0010268821124486056, 'depth': 3, 'l2_leaf_reg': 1.874227876670245}. 
[I 2024-04-25 10:56:30,903] Trial 1 finished with values: {'R2': 0.16930301938532755, 'MAE': 0.12609052783616925, 'RMSE': 0.15397844338762345} and parameters: {'n_estimators': 104, 'learning_rate': 0.00149091529373247, 'depth': 13, 'l2_leaf_reg': 4.1817813820872}. 
[I 2024-04-25 10:56:32,239] Trial 2 finished with values: {'R2': 0.7660796565099182, 'MAE': 0.06038835986565149, 'RMSE': 0.08170950987677891} and parameters: {'n_estimators': 137, 'learning_rate': 0.03274653277662849, 'depth': 9, 'l2_leaf_reg': 1.1027317245783288}. 
[I 2024-04-25 10:56:33,399] Tr

KeyboardInterrupt: 

# Обработка 2022 (2500) (data_2022.csv уже перезаписан)

In [ ]:
df = Dataset(verbose=True)
df.load(join(DATA, 'preprocessed', 'data_2022.csv'), parse_dates=['Дата'])
df.drop(['Unnamed: 0'])

# df.df[df.df['ОН-1/2,P отс. ,кгс/см²'].isin([0])].iloc[:,14:]

alerts = ['2022-05-17 10:00:00', '2022-06-06 08:00:00', '2022-07-28 14:00:00', '2022-12-25 12:00:00', '2022-09-03 16:00:00']
df.df[df.df['Дата'].isin(alerts)] # аномальные дни

df.df.drop(df.df[df.df['Дата'].isin(alerts)].index, inplace=True)

# df.time_series(df.cols[1:], time_axis='Дата', filepath=join(PLOTS, 'time_series', "2022_all"))
# df.corr_matrix(df.cols[1:], target='Добыча воды за 2 ч', filepath=join(PLOTS, 'correlation', "2022_all"), show=True)

# Идеи

In [ ]:
cols = df.cols[1:-2]
for col in cols:
    df.df[col] = df.zscore(df.df[col], window=300, thresh=3.0, coeff=1.0)
    # plot(df.df[col], df.df[col + '_cleaned'], max, min)


In [ ]:
cols = ['Water_2hours', 'Vlagomer']
for col in cols:
    df.df[col] = df.zscore(df.df[col], window=300, thresh=3.0, coeff=1.0)
    # plot(df.df[col], df.df[col + '_cleaned'], max, min)
df.corr_matrix(df.cols[1:], target='Vlagomer', show=True)